In [1]:
# STEP 1: Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns
import os

# STEP 2: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
# Load dataset
df = pd.read_csv('/content/drive/MyDrive/ML_work/datafiles/station_data/palam_merged.csv')

# Drop or handle NaNs
if df.isnull().sum().sum() > 0:
    print("⚠️ Dropping rows with NaNs")
    df = df.dropna()

# Define features and target
X = df.drop(columns=['datetime', 'heatwave'], errors='ignore')
y = df['heatwave']

# Train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define models
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score, mean_squared_error

models = {
    "LinearRegression": LinearRegression(),
    "Ridge": Ridge(),
    "Lasso": Lasso(),
    "DecisionTree": DecisionTreeRegressor(random_state=42),
    "RandomForest": RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42),
    "GradientBoosting": GradientBoostingRegressor(n_estimators=100, random_state=42),
    "SVR": SVR(),
    "KNN": KNeighborsRegressor()
}

# Evaluate all models
results = []
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    cv_r2 = cross_val_score(model, X, y, cv=5, scoring='r2').mean()
    results.append({
        "Model": name,
        "R2 Score": round(r2, 6),
        "RMSE": round(rmse, 6),
        "CV R2 Mean": round(cv_r2, 6)
    })

# Print results
import pandas as pd
results_df = pd.DataFrame(results).sort_values(by="R2 Score", ascending=False)
print("📊 Model Comparison:\n", results_df)


⚠️ Dropping rows with NaNs
📊 Model Comparison:
               Model  R2 Score      RMSE  CV R2 Mean
3      DecisionTree  1.000000  0.000000    1.000000
4      RandomForest  1.000000  0.000000    1.000000
5  GradientBoosting  1.000000  0.000013    1.000000
7               KNN  0.881718  0.164691    0.829098
1             Ridge  0.639224  0.287626    0.538763
0  LinearRegression  0.639209  0.287632    0.538694
6               SVR  0.428695  0.361946    0.368223
2             Lasso  0.392385  0.373271    0.380098


In [3]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import r2_score, mean_squared_error
import matplotlib.pyplot as plt

# Load preprocessed data
df = pd.read_csv('/content/drive/MyDrive/ML_work/datafiles/station_data/palam_merged.csv')

# Drop unnecessary columns
df.drop(columns=['datetime', 'hour'], inplace=True, errors='ignore')

# Drop rows with missing values
df.dropna(inplace=True)

# Split into features and target
X = df.drop(columns='heatwave')
y = df['heatwave']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 🔧 Random Forest with regularization (reduced complexity)
rf = RandomForestRegressor(
    n_estimators=100,
    max_depth=5,              # ✅ limit depth to prevent overfitting
    min_samples_leaf=5,       # ✅ require more samples per leaf
    max_features='sqrt',      # ✅ use fewer features per split
    random_state=42
)

# Train the model
rf.fit(X_train, y_train)

# Predictions
y_pred = rf.predict(X_test)

# Evaluation
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
cv_r2 = cross_val_score(rf, X, y, cv=5, scoring='r2').mean()

# Print results
print(f"📌 R² Score: {r2:.6f}")
print(f"📌 RMSE: {rmse:.6f}")
print(f"📌 Cross-validated R²: {cv_r2:.6f}")


📌 R² Score: 0.981250
📌 RMSE: 0.065570
📌 Cross-validated R²: 0.975322


In [4]:
import joblib
model_path = "/content/drive/MyDrive/Colab Notebooks/saved_models/rf_palam.pkl"
joblib.dump(rf, model_path)

print(f"✅ Palam RF model saved to: {model_path}")


✅ Palam RF model saved to: /content/drive/MyDrive/Colab Notebooks/saved_models/rf_palam.pkl
